In [ ]:
#!pip install google-generative-ai


ERROR: Could not find a version that satisfies the requirement google-generative-ai (from versions: none)
ERROR: No matching distribution found for google-generative-ai


In [1]:
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini API
genai.configure(api_key=userdata.get('GENAI_API'))

def rag_query(query_text, n_results=3, max_chars_per_doc=1000):
    """
    Fonction RAG : récupère les CVs via Chroma et génère une réponse avec Gemini.
    
    Args:
        query_text (str): La question ou requête utilisateur
        n_results (int): Nombre de documents CV à récupérer
        max_chars_per_doc (int): Nombre max de caractères à prendre par document pour le contexte
    Returns:
        str: Réponse générée par le LLM basée sur les CVs
    """

    # Étape 1 : récupérer les documents via Chroma
    results = simple_query(query_text, n_results=n_results)
    
    if not results or not results.get("documents") or not results["documents"][0]:
        return "❌ Aucun CV pertinent trouvé pour cette requête."

    # Étape 2 : construire le contexte à partir des extraits
    documents = results["documents"][0]
    metadatas = results["metadatas"][0]
    
    context_chunks = []
    for doc, meta in zip(documents, metadatas):
        preview = doc[:max_chars_per_doc] + ("..." if len(doc) > max_chars_per_doc else "")
        meta_info = f"👤 Candidate: {meta.get('candidate_name', 'Unknown')}, " \
                    f"📍 Location: {meta.get('location', 'N/A')}, " \
                    f"💼 Seniority: {meta.get('seniority_level', 'N/A')}"
        context_chunks.append(f"{meta_info}\n{preview}")
    
    context = "\n\n---\n\n".join(context_chunks)

    # Étape 3 : construire le prompt pour le LLM
    prompt = f"""
Vous êtes un assistant RH. Voici des extraits pertinents de CVs correspondant à la requête :
{context}

Répondez de manière concise et informative à la question suivante, en vous basant uniquement sur ces CVs :
{query_text}
"""
    
    # Étape 4 : génération via Gemini
    response = genai.chat.create(
        model="models/gemini-1.5-chat",
        messages=[{"role": "user", "content": prompt}],
    )

    return response.last.message.content

ModuleNotFoundError: No module named 'google.generativeai'

In [ ]:
# Exemple d'utilisation
#query = "Cherche un développeur Python avec expérience en machine learning et data science."
query = "les punitions le plus important dans le code pénal."
answer = rag_query(query)
print("💡 Réponse RAG :")
print(answer)